# Code Archive

This notebook contains experiments with training various autoencoders, as well as using code for gradient decent from lecture.


None of this code is used in the final product, as we not only moved model training to the URI ssh GPU server, but we also changed the tragectory of the project to classification rather than image generation.

Training an autoencoder using thousands of high quality images takes far too long to be done on our local machines, so the output of most of these cells is not displayed.

## NOTE: all of these models, besides code used from lectures, was generated using ChatGPT.

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:

# Load and preprocess data
# Assuming images are in a directory structure and are resized to 64x64 for simplicity
image_size = (64, 64)
batch_size = 32

# Function to load and preprocess images from a directory
def load_images_from_directory(directory):
    images = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):  # Adjust file extensions as needed
            img_path = os.path.join(directory, filename)
            img = load_img(img_path, target_size=image_size)
            img_array = img_to_array(img) / 255.0  # Preprocess image
            images.append(img_array)
    return np.array(images)

# Load image data
data_dir = '/content/drive/MyDrive/ML final project/datasets/CLASSES_400_300_Part2/'
train_data = load_images_from_directory(data_dir)

# Define the autoencoder
def build_autoencoder(input_shape=(64, 64, 3), latent_dim=32):
    # Encoder
    encoder_input = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoder_input)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Flatten()(x)
    latent = Dense(latent_dim, activation='relu')(x)

    # Decoder
    x = Dense(8 * 8 * 128, activation='relu')(latent)
    x = Reshape((8, 8, 128))(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoder_output = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    # Autoencoder
    autoencoder = Model(encoder_input, decoder_output)
    encoder = Model(encoder_input, latent)  # separate model for extracting latent vectors

    return autoencoder, encoder

# Build and compile the model
latent_dim = 32
autoencoder, encoder = build_autoencoder(input_shape=(64, 64, 3), latent_dim=latent_dim)
autoencoder.compile(optimizer='adam', loss='mse')

# Train the autoencoder
epochs = 50
autoencoder.fit(train_data, train_data, epochs=epochs, batch_size = batch_size)

# Save models
autoencoder.save('/content/drive/MyDrive/ML final project/datasets/autoencoder_model.h5')
encoder.save('/content/drive/MyDrive/ML final project/datasets/encoder_model.h5')

# To encode images into latent vectors (color representation)
def encode_images(images):
    return encoder.predict(images)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Load and preprocess data
# Assuming images are in a directory structure and are resized to 64x64 for simplicity
image_size = (64, 64)
batch_size = 32

datagen = ImageDataGenerator(rescale=1./255)
train_data = datagen.flow_from_directory(
    '/content/drive/MyDrive/ML final project/datasets/CLASSES_400_300_Part2/',
    target_size=image_size,
    batch_size=batch_size,
    class_mode=None,
    color_mode='rgb'
)

# Define the autoencoder
def build_autoencoder(input_shape=(64, 64, 3), latent_dim=32):
    # Encoder
    encoder_input = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoder_input)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Flatten()(x)
    latent = Dense(latent_dim, activation='relu')(x)

    # Decoder
    x = Dense(8 * 8 * 128, activation='relu')(latent)
    x = Reshape((8, 8, 128))(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoder_output = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    # Autoencoder
    autoencoder = Model(encoder_input, decoder_output)
    encoder = Model(encoder_input, latent)  # separate model for extracting latent vectors

    return autoencoder, encoder

# Build and compile the model
latent_dim = 32
autoencoder, encoder = build_autoencoder(input_shape=(64, 64, 3), latent_dim=latent_dim)
autoencoder.compile(optimizer='adam', loss='mse')


In [ ]:
import pandas as pd
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import numpy as np

In [ ]:
#GRADIENT DECENT TEST (FROM LECTURE)
#(1)
# Set paths
csv_file = '/content/drive/MyDrive/ML final project/datasets/iris_labelsShort.csv'  # Path to CSV file
root_dir = '/content/drive/MyDrive/ML final project/datasets/CLASSES_400_300_Part2/'  # Directory containing the .tiff files

# Define transformations (e.g., resizing and normalization)
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize all images to a common size
    transforms.ToTensor(),  # Convert images to PyTorch tensors
])

# Load dataset
iris_dataset = IrisDataset(csv_file=csv_file, root_dir=root_dir, transform=transform)

# Split into training and test datasets (80/20 split)
train_size = int(0.8 * len(iris_dataset))
test_size = len(iris_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(iris_dataset, [train_size, test_size])

In [ ]:
#(2)
# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Dataset statistics
train_labels = [label for _, label in train_dataset]
test_labels = [label for _, label in test_dataset]

print(f'Training datapoints: {len(train_dataset)}, Test datapoints: {len(test_dataset)}')
print('Image size:', iris_dataset[0][0].size())
print(f'Classes: {set(iris_dataset.data.iloc[:, 1])}')
#print(f'Class distribution (train): {np.bincount(train_labels)}')
#print(f'Class distribution (test): {np.bincount(test_labels)}')

Training datapoints: 2350, Test datapoints: 588
Image size: torch.Size([3, 128, 128])
Classes: {'gray', 'brown', 'hazel', 'green', 'blue'}


In [ ]:

# flatten the data into shape (N, D) where N is the number of samples and D is the dimensionality of the data
Xtr = train_dataset.data.reshape(-1, 128*128)
# add a column of ones to the data to account for the bias term
Xtr = torch.hstack((torch.ones(Xtr.shape[0],1), Xtr))
# normalize the data to have pixel values between 0 and 1
Xtr = Xtr / 255

# apply the same transformations to the test data
Xte = test_dataset.data.reshape(-1, 28*28)
Xte = torch.hstack((torch.ones(Xte.shape[0],1), Xte))
Xte = Xte / 255

# extract the labels
Ytr = train_dataset.targets
Yte = test_dataset.targets

print(f'Xtr shape: {Xtr.shape} Ytr shape: {Ytr.shape}')
print(f'Xte shape: {Xte.shape} Yte shape: {Yte.shape}')

AttributeError: 'Subset' object has no attribute 'data'

In [ ]:
#(3)
!pip install torcheval
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import MNIST
from torcheval.metrics.functional import multiclass_accuracy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 3.3 MB/s eta 0:00:00


In [ ]:
#(4)
# Flatten the data into shape (N, D) where N is the number of samples, and D is the dimensionality of the data
def flatten_and_normalize(dataset, image_size=(128, 128)):
    # Extract images and labels from the dataset
    images = []
    labels = []
    labels_mapping = {'gray': 0, 'brown': 1, 'hazel': 2, 'green': 3, 'blue': 4}
    for i in range(len(dataset)):
        img, label = dataset[i]  # Access image and label
        #print(img, label)
        if not isinstance(img, torch.Tensor):  # Ensure the image is a tensor
            img = transforms.ToTensor()(img)  # Convert to tensor if needed

        images.append(img.view(-1))  # Flatten the image and add to the list
        labels.append(label)

    # Stack images into a single tensor
    images = torch.stack(images)
    #labels.append(labels_mapping.get(label))
    labels_numeric = [labels_mapping.get(label, -1) for label in labels]
    labels_tensor = torch.tensor(labels_numeric, dtype=torch.long)
    # Normalize the data to have pixel values between 0 and 1
    images = images / 255.0

    # Add a column of ones to the data to account for the bias term
    images = torch.hstack((torch.ones(images.shape[0], 1), images))

    return images, labels_tensor

# Apply transformations to training and test data
Xtr, Ytr = flatten_and_normalize(train_dataset, image_size=(128, 128))
Xte, Yte = flatten_and_normalize(test_dataset, image_size=(128, 128))

# Print shapes
print(f'Xtr shape: {Xtr.shape}, Ytr shape: {Ytr.shape}')
print(f'Xte shape: {Xte.shape}, Yte shape: {Yte.shape}')

Xtr shape: torch.Size([2350, 49153]), Ytr shape: torch.Size([2350])
Xte shape: torch.Size([588, 49153]), Yte shape: torch.Size([588])


In [ ]:
#(5)
# define hyperparameters
n_epochs = 100
learning_rate = 2e-3

# initialize a random weight matrix
# every row in the matrix is a weight vector for a single class, as we have 10 classes, we have 10 rows
W = torch.rand(5, Xtr.shape[1]) / torch.sqrt(torch.tensor(Xtr.shape[1]))

# TRAINING LOOP
for epoch in range(n_epochs):
    # FORWARD PASS
    # first we compute the logits
    logits = Xtr @ W.T
    # then we compute the probabilities by applying the softmax function
    # we can use the softmax function from torch.nn.functional or implement it ourselves
    # the advantage of using the torch function is that it is numerically stable
    # i.e., it avoids numerical issues that can occur when the values in the logits
    # are very large or very small
    probs = F.softmax(logits, dim=1)
    # the shape of `probs` is (N, C) where N is the number of samples and C is
    # the number of classes. every row in this matrix is a probability distribution
    # over the classes for a single datapoint
    # we can use the multi_class_accuracy function from torcheval to compute the accuracy
    accuracy = multiclass_accuracy(probs, Ytr)
    # we can use the cross entropy loss function from torch.nn.functional to compute the loss
    # the function expects the predicted unnormalized logits and the true labels
    # it returns the average loss over the samples
    loss = F.cross_entropy(logits, Ytr, reduction='mean')
    # BACKWARD PASS
    # first we calculate the partial derivetives of the loss with respect to the weights
    y_one_hot = F.one_hot(Ytr, 5)
    dloss = Xtr.T @ (probs - y_one_hot)
    # then we update the weights using the gradient and the learning rate
    W = W - learning_rate * dloss.T
    # log training progress
    print(f'Epoch {epoch+1}/{n_epochs}, Loss: {loss.item():.6f}, Accuracy: {100*accuracy.item():.2f}%')



Epoch 1/100, Loss: 1.608022, Accuracy: 75.11%
Epoch 2/100, Loss: 0.838293, Accuracy: 80.51%
Epoch 3/100, Loss: 0.759129, Accuracy: 80.51%
Epoch 4/100, Loss: 0.748036, Accuracy: 80.51%
Epoch 5/100, Loss: 0.745927, Accuracy: 80.51%
Epoch 6/100, Loss: 0.744398, Accuracy: 80.51%
Epoch 7/100, Loss: 0.743229, Accuracy: 80.51%
Epoch 8/100, Loss: 0.742309, Accuracy: 80.51%
Epoch 9/100, Loss: 0.741563, Accuracy: 80.51%
Epoch 10/100, Loss: 0.740946, Accuracy: 80.51%
Epoch 11/100, Loss: 0.740423, Accuracy: 80.51%
Epoch 12/100, Loss: 0.739972, Accuracy: 80.51%
Epoch 13/100, Loss: 0.739576, Accuracy: 80.51%
Epoch 14/100, Loss: 0.739224, Accuracy: 80.51%
Epoch 15/100, Loss: 0.738906, Accuracy: 80.51%
Epoch 16/100, Loss: 0.738616, Accuracy: 80.51%
Epoch 17/100, Loss: 0.738348, Accuracy: 80.51%


KeyboardInterrupt: 

In [ ]:
import torch.nn as nn

class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create an instance of the model
model = SimpleNN(input_size=Xtr.shape[1], hidden_size=128, output_size=10)

NameError: name 'Xtr' is not defined

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
batch_size = 64
learning_rate = 0.001
num_epochs = 10

"""
# Data transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])
"""

# Set paths
csv_file = '/content/drive/MyDrive/ML final project/datasets/iris_labelsShort.csv'  # Path to CSV file
root_dir = '/content/drive/MyDrive/ML final project/datasets/CLASSES_400_300_Part2/'  # Directory containing the .tiff files

# Define transformations (e.g., resizing and normalization)
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize all images to a common size
    transforms.ToTensor(),  # Convert images to PyTorch tensors
])

# Load dataset
iris_dataset = IrisDataset(csv_file=csv_file, root_dir=root_dir, transform=transform)

# Split into training and test datasets (80/20 split)
train_size = int(0.8 * len(iris_dataset))
test_size = len(iris_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(iris_dataset, [train_size, test_size])
#(2)
# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Dataset statistics
#train_labels = [label for _, label in train_dataset]
#test_labels = [label for _, label in test_dataset]
## Load MNIST dataset
#train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
#test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

#train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
#test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Define the model
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(128 * 128 * 3, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 5)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the image
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

model = NeuralNet().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
#print(train_loader.labels[:10])
# Training function
def train():
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch_idx, (images, labels) in enumerate(train_loader):
            #images, labels = images.to(device), labels.to(device)
            #images, labels = batch  # Unpack the image and label tensors
            print(images, labels)
            images = images.to(device)  # Move image to the appropriate device
            #images = torch.tensor(images).to(device)

            labels_mapping = {'gray': 0, 'brown': 1, 'hazel': 2, 'green': 3, 'blue': 4}
            # Convert string labels to numerical labels for the entire batch
            numerical_labels = [labels_mapping.get(label, -1) for label in labels]
            # -1 is used as a default value if the label is not found
            labels = torch.tensor(numerical_labels, dtype=torch.long).to(device)
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

# Evaluation function
def evaluate():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Accuracy: {100 * correct / total:.2f}%")

# Save model function
def save_model(path="mnist_model.pth"):
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")

# Main script
if __name__ == "__main__":
    print("Starting training...")
    train()
    print("Evaluating the model...")
    evaluate()
    save_model()


Starting training...
image label


AttributeError: 'str' object has no attribute 'to'

In [ ]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
batch_size = 64
learning_rate = 0.001
num_epochs = 15
num_classes = 5

"""
# Transformations for the images
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize to 128x128
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize
])
"""
# Custom Dataset
class IrisDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (str): Path to the CSV file with image paths and labels.
            root_dir (str): Directory containing all the images.
            transform (callable, optional): Optional transforms to be applied to images.
        """
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.label_map = {"gray": 0, "brown": 1, "hazel": 2, "green": 3, "blue": 4}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.data.iloc[idx, 0])
        image = Image.open(img_path).convert("RGB")  # Load image as RGB
        label = self.label_map[self.data.iloc[idx, 1]]

        if self.transform:
            image = self.transform(image)
        label = torch.tensor(label, dtype=torch.long)
        return image, label


class EyeDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.label_map = {"gray": 0, "brown": 1, "hazel": 2, "green": 3, "blue": 4}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.data.iloc[idx, 0])
        image = Image.open(img_name).convert("RGB")
        label = self.label_map[self.data.iloc[idx, 1]]

        if self.transform:
            image = self.transform(image)

        return image, label

# Set paths
csv_file = '/content/drive/MyDrive/ML final project/datasets/iris_labelsShort.csv'  # Path to CSV file
root_dir = '/content/drive/MyDrive/ML final project/datasets/CLASSES_400_300_Part2/'  # Directory containing the .tiff files

# Define transformations (e.g., resizing and normalization)
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize all images to a common size
    transforms.ToTensor(),  # Convert images to PyTorch tensors
])

# Load dataset
iris_dataset = IrisDataset(csv_file=csv_file, root_dir=root_dir, transform=transform)

# Split into training and test datasets (80/20 split)
train_size = int(0.8 * len(iris_dataset))
test_size = len(iris_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(iris_dataset, [train_size, test_size])
#(2)
# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

"""
# Load the dataset
train_dataset = EyeDataset(csv_file="train_labels.csv", img_dir="train_images", transform=transform)
test_dataset = EyeDataset(csv_file="test_labels.csv", img_dir="test_images", transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
"""
# Define the model
class EyeClassifier(nn.Module):
    def __init__(self, num_classes):
        super(EyeClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 32 * 32, 128)  # Adjust dimensions if input size changes
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(-1, 64 * 32 * 32)  # Flatten
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model = EyeClassifier(num_classes=num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training function
def train():
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            print(labels)
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

# Evaluation function
def evaluate():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Accuracy: {100 * correct / total:.2f}%")

# Save model function
def save_model(path="eye_classifier.pth"):
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")

# Main script
if __name__ == "__main__":
    print("Starting training...")
    train()
    print("Evaluating the model...")
    evaluate()
    #save_model()

Starting training...
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 4, 1, 3,
        1, 1, 4, 1, 3, 1, 1, 2])
tensor([4, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 4, 2, 1, 1, 1, 1])
tensor([1, 1, 1, 4, 1, 1, 1, 1, 4, 1, 1, 2, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 3,
        1, 1, 1, 1, 1, 1, 1, 4])
tensor([1, 1, 1, 2, 1, 0, 0, 1, 4, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 4, 1, 1])
tensor([1, 1, 1, 1, 3, 4, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1,
        1, 2, 1, 1, 1, 1, 4, 4])
tensor([1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 2, 2, 2, 4, 1, 1, 2, 1,
        1, 1, 2, 1, 2, 1, 3, 1])
tensor([1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 0, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2,
        1, 1, 1, 1, 3, 1, 1, 1])
tensor([1, 4, 3, 1, 1, 1, 1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 1, 4, 1, 4, 1, 2, 3,
        1, 1, 1, 1, 1, 1, 1, 3])
tensor([1, 2, 1, 4, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1